In [1]:
import glob
import time
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("tpch") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/tpch") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/tpch") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
    .config("spark.executor.memory", "2g")\
    .getOrCreate()


In [3]:
db_name = "tpch"

In [4]:
json_files = []

In [5]:
for file in glob.glob("*.json"):
    json_files.append(file)

In [6]:
def populate_db(file_name):
    file_name_without_extension = file_name.split('.')[0]
    df = spark.read.json(file_name, multiLine = "true")
    df.write.format("mongo").mode("append").option("database",db_name).option("collection", file_name_without_extension).save()

In [7]:
strat_time = start_func_time =  end_time = time.time()
for file in json_files:
    strat_time = time.time()
    populate_db(file)
    end_time = time.time()
    print("Time taken to populate %s Collection: %.2f s" %(file.split(".")[0], end_time - strat_time))
end_time = time.time()

print("Total time taken: %.2f s" %(end_time - start_func_time))

Time taken to populate CUSTOMER collection: 4.21 s
Time taken to populate LINEITEM collection: 17.57 s
Time taken to populate NATION collection: 0.28 s
Time taken to populate ORDERS collection: 3.07 s
Time taken to populate PART collection: 0.66 s
Time taken to populate PARTSUPP collection: 1.34 s
Time taken to populate REGION collection: 0.19 s
Time taken to populate SUPPLIER collection: 0.25 s
Total time taken: 27.58 s


In [6]:
spark.stop()